In [1]:
import requests
from bs4 import BeautifulSoup
import json
import random
import time
import pandas as pd

base ="https://www.halfbakedharvest.com/sitemap_index.xml"
header = {"User-Agent": "ingredient_bot/1.0 (+mailto:jakehowell@duck.com; for educational purposes)"}

In [2]:
def parseXML(url):
    site_mapXML = requests.get(url, headers = header,)

    soup = BeautifulSoup(site_mapXML.text, "xml")

    site_maps = []
    for loc_tag in soup.find_all("loc"):
        loc_tag = loc_tag.get_text(strip = True)
        if loc_tag[-3:] == "jpg":
            continue
        elif loc_tag[-3:] != "jpg" and loc_tag not in site_maps:
            site_maps.append(loc_tag)
    return site_maps

In [3]:
site_map = parseXML(base)
print(site_map)


['https://www.halfbakedharvest.com/post-sitemap.xml', 'https://www.halfbakedharvest.com/post-sitemap2.xml', 'https://www.halfbakedharvest.com/post-sitemap3.xml', 'https://www.halfbakedharvest.com/post-sitemap4.xml', 'https://www.halfbakedharvest.com/post-sitemap5.xml', 'https://www.halfbakedharvest.com/page-sitemap.xml', 'https://www.halfbakedharvest.com/web-story-sitemap.xml', 'https://www.halfbakedharvest.com/category-sitemap.xml', 'https://www.halfbakedharvest.com/video-sitemap.xml']


In [4]:
links = []
links.append(parseXML(site_map[1]))
len(links[0])
first_ten_links = links[0][0:10]
first_ten_links

['https://www.halfbakedharvest.com/strawberry-margarita-cream-filled-donuts/',
 'https://www.halfbakedharvest.com/teriyaki-chicken-sub-sandwiches/',
 'https://www.halfbakedharvest.com/tortilla-chip-crusted-chicken-salad-with-avocado-chipotle-lime-dressing-and-queso-fresco/',
 'https://www.halfbakedharvest.com/margarita-chicken-quesadilla-with-margarita-guacamole/',
 'https://www.halfbakedharvest.com/crunchy-oatmeal-peanut-butter-oats-n-honey-bars/',
 'https://www.halfbakedharvest.com/brown-butter-crepe-ice-cream-tacos/',
 'https://www.halfbakedharvest.com/whipped-feta-and-roasted-jalapeno-greek-pizza/',
 'https://www.halfbakedharvest.com/jerk-chicken-with-fired-rice-and-grilled-pineapple-salsa/',
 'https://www.halfbakedharvest.com/chocolate-peanut-butter-swirl-fudge-brownie-ice-cream/',
 'https://www.halfbakedharvest.com/cinnamon-crunch-bagels/']

In [ ]:
df = pd.read_csv("output.csv")
df.head()

,url,recipe_name,ingredients


In [6]:
s = requests.Session()
counter = 0

for url in links[0]:
    if url in set(df['url'].values):
        print(f"Url: {url}, already exists in dataset")
        continue
    resp = s.get(url, headers = header)
    soup = BeautifulSoup(resp.text, "html.parser")

    scripts = soup.find("script", type = "application/ld+json")
    data = json.loads(scripts.string)

    all_recipe_data = data["@graph"]
    for obj in all_recipe_data:
        if obj["@type"] == "Recipe":
            recipe = obj
    ingredients = recipe["recipeIngredient"]

    name = all_recipe_data[0]["headline"]
    
    if url not in df['url']: 
        df.loc[len(df)] = [url, name, ingredients]

    resp.raise_for_status()  # raise for 4xx/5xx
    print(f"Success: {url} (status {resp.status_code})")

    counter += 1
    if counter > 10:
        break
    else:
        print(f"{10 - counter} more links.")

    delay = random.uniform(1, 5)
    print(f"Sleeping {delay:.2f}s before next request...")
    time.sleep(delay)



Success: https://www.halfbakedharvest.com/strawberry-margarita-cream-filled-donuts/ (status 200)
9 more links.
Sleeping 2.31s before next request...
Success: https://www.halfbakedharvest.com/teriyaki-chicken-sub-sandwiches/ (status 200)
8 more links.
Sleeping 2.94s before next request...
Success: https://www.halfbakedharvest.com/tortilla-chip-crusted-chicken-salad-with-avocado-chipotle-lime-dressing-and-queso-fresco/ (status 200)
7 more links.
Sleeping 1.38s before next request...
Success: https://www.halfbakedharvest.com/margarita-chicken-quesadilla-with-margarita-guacamole/ (status 200)
6 more links.
Sleeping 2.67s before next request...
Success: https://www.halfbakedharvest.com/crunchy-oatmeal-peanut-butter-oats-n-honey-bars/ (status 200)
5 more links.
Sleeping 3.84s before next request...
Success: https://www.halfbakedharvest.com/brown-butter-crepe-ice-cream-tacos/ (status 200)
4 more links.
Sleeping 3.31s before next request...
Success: https://www.halfbakedharvest.com/whipped-fet

In [ ]:
df.to_csv("output.csv", index= False)